# 🎯 ConvNeXtPose Testing en Kaggle - Modelos L y M

Este notebook evalúa los modelos ConvNeXtPose L y M en Human3.6M Protocol 2.

**Datasets requeridos:**
- Human3.6M Dataset (con S9_ACT2_i6, S11_ACT2_i6, annotations)
- ConvNeXtPose Pre-trained Models (checkpoints .tar)

**GPU recomendada:** T4 x2 o P100

## 📦 PASO 1: Setup Inicial

In [ ]:
# Clonar repositorio
!git clone https://github.com/EstebanCabreraArbizu/ConvNeXtPose.git
%cd ConvNeXtPose

# Verificar versiones
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 🗂️ PASO 2: Configurar Dataset Human3.6M

Usamos enlaces simbólicos para evitar copiar ~30GB de datos.

In [ ]:
# IMPORTANTE: Ajusta este path al nombre real de tu dataset en Kaggle
KAGGLE_DATASET_PATH = '/kaggle/input/human36m-dataset'  # ← CAMBIAR SEGÚN TU DATASET

# Verificar que el dataset existe
import os
if not os.path.exists(KAGGLE_DATASET_PATH):
    print(f"❌ Dataset no encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/
    raise FileNotFoundError("Verifica el nombre del dataset y actualiza KAGGLE_DATASET_PATH")
else:
    print(f"✓ Dataset encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Contenido:")
    !ls {KAGGLE_DATASET_PATH}

In [ ]:
# Ejecutar script de configuración automática
!python setup_kaggle_dataset.py --kaggle-input {KAGGLE_DATASET_PATH} --output /kaggle/working/data

# Configurar variable de entorno
os.environ['CONVNEXPOSE_DATA_DIR'] = '/kaggle/working/data'
print("\n✓ Variable CONVNEXPOSE_DATA_DIR configurada")

In [ ]:
# Verificar que la estructura es correcta
!python setup_kaggle_dataset.py --verify /kaggle/working/data

## 🎯 PASO 3: Preparar Checkpoints

Extraer los modelos pre-entrenados.

In [ ]:
# IMPORTANTE: Ajusta según el nombre de tu dataset de modelos
MODELS_DATASET_PATH = '/kaggle/input/convnextpose-models'  # ← CAMBIAR SEGÚN TU DATASET

# Verificar modelos disponibles
if os.path.exists(MODELS_DATASET_PATH):
    print("📦 Modelos disponibles:")
    !ls -lh {MODELS_DATASET_PATH}/models_tar/
else:
    print(f"❌ Dataset de modelos no encontrado: {MODELS_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/

In [ ]:

import os
import zipfile
# Crear directorio para modelos
os.makedirs('output/model_dump', exist_ok=True)

#Extraer modelos preentrenados con zipfile (no son realmente archivos tar)

# Extraer modelo L
model_l_path = f'{MODELS_DATASET_PATH}/models_tar/ConvNeXtPose_L.tar'
if os.path.exists(model_l_path):
    print(f"📦 Extrayendo modelo L desde {model_l_path}...")
    with zipfile.ZipFile(model_l_path, 'r') as zip_ref:
        zip_ref.extractall('output/model_dump')
    print(f"✓ Modelo L extraído: {zip_ref.namelist()}")
else:
    print(f"⚠️  Modelo L no encontrado en {model_l_path}")

# Extraer modelo M (opcional)
model_m_path = f'{MODELS_DATASET_PATH}/models_tar/ConvNeXtPose_M.tar'
if os.path.exists(model_m_path):
    print(f"📦 Extrayendo modelo M desde {model_m_path}...")
    with zipfile.ZipFile(model_m_path, 'r') as zip_ref:
        zip_ref.extractall('output/model_dump')
    print(f"✓ Modelo M extraído: {zip_ref.namelist()}")

# Verificar checkpoints extraídos
print("\n📂 Checkpoints disponibles:")
!ls -lh output/model_dump/

In [ ]:
# Verificar el nombre del checkpoint y renombrar si es necesario
import glob

checkpoints = glob.glob('output/model_dump/snapshot_*.pth*')
if checkpoints:
    for ckpt in checkpoints:
        print(f"✓ Checkpoint encontrado: {os.path.basename(ckpt)}")
else:
    print("❌ No se encontraron checkpoints")
    print("Verifica que la extracción fue exitosa")

# Si necesitas renombrar (ej: snapshot_68.pth -> snapshot_70.pth):
# os.rename('output/model_dump/snapshot_68.pth', 'output/model_dump/snapshot_70.pth')

## 🚀 PASO 4: Ejecutar Testing

### Modelo L (Large)

In [ ]:
%cd main

# AJUSTA EL EPOCH según el nombre de tu checkpoint
CHECKPOINT_EPOCH = 68  # Cambiar si tu checkpoint tiene otro número

# Testing modelo L
!python test.py --gpu 0 --epochs {CHECKPOINT_EPOCH} --variant L

### Modelo M (Medium) - Opcional

In [ ]:
# Si tienes el checkpoint del modelo M, ejecuta esto:
# !python test.py --gpu 0 --epochs {CHECKPOINT_EPOCH} --variant M

## 📊 PASO 5: Verificar Resultados

In [ ]:
# Ver resultados generados
%cd ..
!ls -lh output/result/

# Leer log de resultados
import glob
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"\n📄 Últimas líneas del log ({os.path.basename(latest_log)}):")
    !tail -n 20 {latest_log}
else:
    print("⚠️  No se encontraron logs")

## 📈 PASO 6: Análisis de Resultados

In [ ]:
import re

def extract_mpjpe_from_log(log_path):
    """Extrae el MPJPE del log"""
    with open(log_path, 'r') as f:
        content = f.read()
    
    # Buscar patrón de MPJPE
    pattern = r'MPJPE.*?(\d+\.\d+)'
    matches = re.findall(pattern, content)
    
    if matches:
        return float(matches[-1])  # Último valor
    return None

# Extraer resultados
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    mpjpe = extract_mpjpe_from_log(latest_log)
    
    print("\n" + "="*60)
    print("  📊 RESULTADOS FINALES")
    print("="*60)
    
    if mpjpe:
        print(f"\n  MPJPE (Protocol 2): {mpjpe:.2f} mm")
        
        # Comparar con paper
        expected = {
            'L': 42.3,
            'M': 44.6
        }
        
        # Determinar variante
        for variant, expected_val in expected.items():
            diff = abs(mpjpe - expected_val)
            if diff < 5:
                print(f"\n  Variante detectada: {variant}")
                print(f"  Valor del paper: {expected_val:.2f} mm")
                print(f"  Diferencia: {mpjpe - expected_val:+.2f} mm")
                
                if diff < 2:
                    print("  ✅ Resultado excelente (dentro de ±2mm)")
                elif diff < 5:
                    print("  ✓ Resultado aceptable (dentro de ±5mm)")
                break
    else:
        print("\n⚠️  No se pudo extraer MPJPE del log")
        print("Revisa el log manualmente en output/log/")
    
    print("\n" + "="*60)
else:
    print("❌ No se encontraron logs de testing")

## 💾 PASO 7: Guardar Outputs

Kaggle guarda automáticamente todo en `/kaggle/working/`. Opcionalmente puedes copiar resultados específicos:

In [ ]:
# Crear resumen de resultados
import json
from datetime import datetime

summary = {
    'timestamp': datetime.now().isoformat(),
    'model': 'ConvNeXtPose-L',  # Cambiar según modelo testeado
    'checkpoint_epoch': CHECKPOINT_EPOCH,
    'dataset': 'Human3.6M Protocol 2',
    'mpjpe_mm': mpjpe if 'mpjpe' in locals() else None,
    'pytorch_version': torch.__version__,
    'cuda_version': torch.version.cuda if torch.cuda.is_available() else None
}

# Guardar resumen
with open('output/result/test_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Resumen guardado en output/result/test_summary.json")
print("\n📄 Contenido:")
print(json.dumps(summary, indent=2))

## 🎉 Testing Completado!

Los resultados están en:
- **Logs**: `output/log/`
- **Resultados**: `output/result/`
- **Resumen JSON**: `output/result/test_summary.json`

Todos los archivos en `/kaggle/working/ConvNeXtPose/output/` se guardarán automáticamente cuando hagas commit del notebook.